In [ ]:
df = pd.read_csv("Crop_recommendation.csv")
crop_means = df.groupby("label")[["N", "P", "K", "rainfall", "ph", "humidity", "temperature"]].mean()
column_names = crop_means.columns
area_sqft = 10000


def water_amount(area_sqft, rainfall_deficit_mm):
    area_m2 = area_sqft * 0.0929  # square feet to square meters
    water_liters = rainfall_deficit_mm * area_m2  # mm deficit to liters
    return water_liters

def soil_amendment_recommendations(crop, current_values, column_names, area_sqft):
    crop = crop.lower()
    if crop not in crop_means.index:
        return f"Crop '{crop}' not found in dataset."
    
    crop_avgs = crop_means.loc[crop]
    differences = crop_avgs - current_values
    
    # Create a dictionary to map column indices to their names
    value_dict = {name: (current, optimal, diff) for name, current, optimal, diff in 
                  zip(column_names, current_values, crop_avgs, differences)}
    
    # Define thresholds for significant deviations
    thresholds = {
        'N': 10,        # Nitrogen threshold
        'P': 5,         # Phosphorus threshold
        'K': 5,         # Potassium threshold
        'rainfall': 50, # Rainfall threshold in mm
        'ph': 0.5,      # pH threshold
        'humidity': 10, # Humidity threshold
        'temperature': 2 # Temperature threshold
    }
    
    recommendations = []
    environment_factors = []
    soil_factors = []
    
    # Check soil nutrients (N, P, K)
    for nutrient in ['N', 'P', 'K']:
        current, optimal, diff = value_dict[nutrient]
        if abs(diff) > thresholds[nutrient]:
            if diff > 0:
                soil_factors.append(f"The soil is deficient in {nutrient}. Current level is {current:.3f}, but optimal level for {crop} is around {optimal:.3f}. Consider adding {nutrient}-rich fertilizer to increase by approximately {abs(diff):.3f} units.")
            else:
                soil_factors.append(f"The soil has excess {nutrient}. Current level is {current:.3f}, but optimal level for {crop} is around {optimal:.3f}. Avoid adding more {nutrient} fertilizers for now.")
        else:
            soil_factors.append(f"The {nutrient} level is within acceptable range for {crop} cultivation (current: {current:.3f}, optimal: {optimal:.3f}).")
    
    # Check pH
    current, optimal, diff = value_dict['ph']
    if abs(diff) > thresholds['ph']:
        if diff > 0:
            soil_factors.append(f"The soil pH is too acidic at {current:.3f}. Optimal pH for {crop} is around {optimal:.3f}. Consider adding agricultural lime to raise the pH by approximately {abs(diff):.3f} units.")
        else:
            soil_factors.append(f"The soil pH is too alkaline at {current:.3f}. Optimal pH for {crop} is around {optimal:.3f}. Consider adding sulfur or organic matter to lower the pH by approximately {abs(diff):.3f} units.")
    else:
        soil_factors.append(f"The soil pH of {current:.3f} is within optimal range for {crop} cultivation (optimal: {optimal:.3f}).")
    
    # Check environmental factors
    for factor in ['rainfall', 'humidity', 'temperature']:
        current, optimal, diff = value_dict[factor]
        if abs(diff) > thresholds[factor]:
            if factor == 'rainfall':
                if diff > 0:
                    environment_factors.append(f"The area receives insufficient rainfall. Current average is {current:.3f}mm, but {crop} typically requires around {optimal:.3f}mm. Consider supplemental irrigation of approximately {abs(water_amount(area_sqft,diff)):.3f}litres")
                else:
                    environment_factors.append(f"The area receives excess rainfall. Current average is {current:.3f}mm, but {crop} typically grows best with around {optimal:.3f}mm. Consider improved drainage systems to manage excess {abs(diff):.3f}mm.")
            elif factor == 'humidity':
                if diff > 0:
                    environment_factors.append(f"The humidity level is too low at {current:.3f}%. Optimal humidity for {crop} is around {optimal:.3f}%. This may increase water requirements by approximately {abs(diff):.3f}%.")
                else:
                    environment_factors.append(f"The humidity level is too high at {current:.3f}%. Optimal humidity for {crop} is around {optimal:.3f}%. This may increase disease risk by approximately {abs(diff):.3f}%.")
            elif factor == 'temperature':
                if diff > 0:
                    environment_factors.append(f"The temperature is too cool at {current:.3f}°C. Optimal temperature for {crop} is around {optimal:.3f}°C. Consider adjusting planting date or using row covers to increase temperature by approximately {abs(diff):.3f}°C.")
                else:
                    environment_factors.append(f"The temperature is too warm at {current:.3f}°C. Optimal temperature for {crop} is around {optimal:.3f}°C. Consider providing shade or adjusting planting date to decrease temperature by approximately {abs(diff):.3f}°C.")
        else:
            environment_factors.append(f"The {factor} level of {current:.3f} is suitable for {crop} cultivation (optimal: {optimal:.3f}).")
    
    # Create final recommendation summary
    summary = f"## Soil Amendment Recommendations for {crop.title()} Cultivation\n\n"
    
    summary += "### Soil Nutrient Analysis\n"
    for rec in soil_factors:
        summary += f"- {rec}\n"
    
    summary += "\n### Environmental Factors\n"
    for rec in environment_factors:
        summary += f"- {rec}\n"
    
    # Overall suitability assessment
    significant_deviations = sum(1 for factor, (current, optimal, diff) in value_dict.items() 
                               if abs(diff) > thresholds.get(factor, 0))
    total_factors = len(value_dict)
    
    if significant_deviations == 0:
        summary += "\n### Overall Assessment\n"
        summary += f"Your conditions are excellent for {crop} cultivation! All measured parameters are within optimal ranges.\n"
    elif significant_deviations <= 2:
        summary += "\n### Overall Assessment\n"
        summary += f"Your conditions are generally favorable for {crop} cultivation with minor adjustments needed as noted above.\n"
    elif significant_deviations <= 4:
        summary += "\n### Overall Assessment\n"
        summary += f"Your conditions require moderate adjustments for optimal {crop} cultivation. Address the factors mentioned above to improve potential yield.\n"
    else:
        summary += "\n### Overall Assessment\n"
        summary += f"Your conditions present significant challenges for {crop} cultivation. Consider implementing the recommendations above or evaluating alternative crops better suited to your conditions.\n"
    
    return summary

# sample input
column_names = ['N', 'P', 'K', 'rainfall', 'ph', 'humidity', 'temperature']
current_values = np.array([85, 45, 38, 160, 6.3, 68, 24])
recommendations = soil_amendment_recommendations("Rice", current_values, column_names, area_sqft)
print(recommendations)